# Creación y entrenamiento de un sistema de preguntas y respuestas utilizando Haystack

**Tabla de contenidos:**

1. [División del dataset](#división-del-dataset)
2. [Importar las bibliotecas necesarias y definir modelos](#Importar-las-bibliotecas-necesarias-y-definir-modelos)
3. [Definición de las funciones auxiliares](#Definición-de-las-funciones-auxiliares)
4. [Preparación de los datos](#Preparación-de-los-datos)
5. [Uso de la pipeline para hacer preguntas](#Uso-de-la-pipeline-para-hacer-preguntas)


## División del dataset

El código proporcionado en el script "json_split.py" toma un archivo JSON de SQuAD como entrada y divide el conjunto de datos en tres partes: entrenamiento, desarrollo y validación, con proporciones de 80%, 10% y 10%, respectivamente. Luego, guarda estos conjuntos divididos en archivos JSON independientes.

- El archivo de entrada es un archivo JSON de SQuAD ubicado en ['./data/data.json'](https://github.com/josuemzx/Chatbot-para-refugiados/blob/main/data/Dataset%20propio/data.json), que se puede encontrar en el repositorio de GitHub [Chatbot-para-refugiados](https://github.com/josuemzx/Chatbot-para-refugiados/).
- La función split_squad_data se encarga de dividir el conjunto de datos en las proporciones especificadas. Esta función acepta un archivo JSON de entrada y proporciones opcionales para entrenamiento y desarrollo (por defecto, 0.8 y 0.1).
- Después de dividir los datos, el código guarda los conjuntos divididos en archivos JSON:
    - El conjunto de entrenamiento se guarda en 'data/train.json'.
    - El conjunto de desarrollo se guarda en 'data/dev.json'.
    - El conjunto de validación se guarda en 'data/val.json'.

Esta división permite entrenar, ajustar y evaluar modelos de aprendizaje automático utilizando diferentes subconjuntos del conjunto de datos original, lo que ayuda a evitar el sobreajuste y a garantizar que el modelo generalice bien a datos no vistos previamente.

In [ ]:
!python json_split.py

## Importar las bibliotecas necesarias y definir modelos

En esta sección, importamos las bibliotecas necesarias para la creación y entrenamiento del sistema de preguntas y respuestas. Además, definimos los modelos pre-entrenados en español que vamos a utilizar.

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import FARMReader, DensePassageRetriever, PreProcessor
from haystack.utils import convert_files_to_docs
from haystack.pipelines import ExtractiveQAPipeline
from typing import List
import os

spanish_models = [
    "deepset/xlm-roberta-base-squad2-distilled",
    "dccuchile/bert-base-spanish-wwm-uncased",
    "dccuchile/albert-base-10-spanish-finetuned-qa-sqac"
]

## Definición de las funciones auxiliares

Aquí definimos las funciones auxiliares que se utilizarán para crear el document store, el retriever, entrenar el reader y crear la pipeline.


In [ ]:
def create_document_store(docs: List[str], custom_db_filename: str) -> FAISSDocumentStore:
    # Crear objeto FAISSDocumentStore y cargar documentos
    # Crear una carpeta si no existe. Aconsejo borrarla si ya existe para que no dé error.
    folder_name = "databases"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    document_store = FAISSDocumentStore(sql_url=f"sqlite:///{folder_name}/{custom_db_filename}")
    document_store.write_documents(docs)
    return document_store

def create_retriever(document_store: FAISSDocumentStore, model: str) -> DensePassageRetriever:
    # Crear objeto DensePassageRetriever
    retriever = DensePassageRetriever(
        document_store=document_store,
        query_embedding_model=model,
        passage_embedding_model=model,
        use_gpu=True
    )
    return retriever

def train_reader(model: str, dataset_path: str, save_dir: str) -> FARMReader:
    # Crear objeto FARMReader para entrenar el modelo DPR con el archivo SQuAD
    reader = FARMReader(
        model_name_or_path=model,
        use_gpu=True,
        context_window_size=100,
        top_k_per_candidate=3,
        return_no_answer=False,
        num_processes=0
    )
    reader.train(
        data_dir=dataset_path,
        train_filename='train.json',
        dev_filename='dev.json',
        test_filename='val.json',
        use_gpu=True,
        batch_size=8,
        n_epochs=25,
        learning_rate=2e-5,
        evaluate_every=80
    )
    return reader

def create_pipeline(reader: FARMReader, retriever: DensePassageRetriever) -> ExtractiveQAPipeline:
    # Crear objeto ExtractiveQAPipeline que use el document_store y el FARMReader entrenado para responder a las preguntas
    pipeline = ExtractiveQAPipeline(
        reader=reader,
        retriever=retriever
    )
    return pipeline

## Preparación de los datos

En esta sección, preparamos los datos para su procesamiento, convirtiendo los archivos en documentos y aplicando un preprocesamiento.


In [ ]:
# Rutas a los archivos y modelo
docs_path = "data/docs/"
dataset_path = "data/"
save_dir = "trained_reader/"

# Convertir archivos en documentos y procesarlos con el PreProcessor
all_docs = convert_files_to_docs(dir_path=docs_path)
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

pipes = []
for model, i in zip(spanish_models, range(len(spanish_models))):

    # Crear document_store y cargar documentos
    document_store = create_document_store(docs, str(i)+'_db')

    # Crear DensePassageRetriever y actualizar embeddings en el document_store
    retriever = create_retriever(document_store, model)
    document_store.update_embeddings(retriever)
    # Crear una carpeta si no existe
    folder_name = "retrievers"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    retriever.save(folder_name+'/'+str(i))

    # Entrenar FARMReader con el conjunto de datos y guardar el modelo
    reader = train_reader(model, dataset_path, save_dir)
    # Crear una carpeta si no existe
    folder_name = "readers"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    reader.save(folder_name+'/'+str(i))
    
    print("Entrenamiento finalizado.")

    # Crear ExtractiveQAPipeline con el FARMReader entrenado y el DensePassageRetriever
    pipeline = create_pipeline(reader, retriever)
    pipes.append(pipeline)

## Uso de la pipeline para hacer preguntas

Finalmente, en esta sección, utilizamos la pipeline para realizar preguntas y obtener respuestas de los documentos procesados.


In [ ]:
# Ahora puedes usar la pipeline para hacer preguntas, por ejemplo:
question = "¿Dónde acudir al llegar a España?"
for pipe in pipes:
    result = pipe.run(query=question)['answers'][0].answer
    print(result)

Este notebook te guía a través de la creación y entrenamiento de un sistema de preguntas y respuestas utilizando el framework Haystack. Con este sistema, puedes buscar información relevante en un conjunto de documentos y obtener respuestas precisas a tus preguntas.
